# Modelo de Recomendación Item-Item

### Librerias

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Cargar archivo

In [18]:
# Cargar el conjunto de datos
steam_games = pd.read_csv('steam_games_cleaned.csv')

### Tratamiento adicional a los datos

In [19]:
# Seleccionar las columnas necesarias
df = steam_games.loc[:, ["specs", "item_id", "name"]]

# Convertir 'item_id' a tipo entero
df["item_id"] = df["item_id"].astype(int)

# Limpiar la columna 'specs'
df['specs'] = df['specs'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace("'", ''))

### Modelo

Creación de un Vectorizador de Texto (CountVectorizer):

Propósito: Convertir texto (en este caso, la columna 'specs') en vectores numéricos.

Proceso: El vectorizador asigna un número a cada palabra única presente en el texto y cuenta su frecuencia. Cada documento (en este caso, cada juego) se representa como un vector donde cada posición corresponde a la frecuencia de una palabra.

Importancia: Esta representación vectorial es necesaria para calcular la similitud del coseno, ya que este método trabaja con vectores numéricos.

In [20]:
# Crear un vectorizador de texto
cv = CountVectorizer()
vectores = cv.fit_transform(df['specs']).toarray()

Cálculo de la Similitud del Coseno entre Vectores:

Propósito: Evaluar la similitud entre los vectores numéricos de diferentes juegos.

Proceso: Se utiliza la métrica de similitud del coseno, que mide el coseno del ángulo entre dos vectores. Cuanto más cercano a 1, más similares son los vectores.

Importancia: La similitud del coseno es clave para determinar qué tan parecidos son los juegos entre sí. Esto se utiliza para generar recomendaciones, ya que los juegos con vectores similares son considerados como recomendaciones potenciales.

In [21]:
# Calcular la similitud del coseno entre vectores
similitud = cosine_similarity(vectores)

A continuación, se genera una función (recomendacion) que toma como entrada el ID de un juego y devuelve una lista de los cinco juegos más recomendados basándose en la similitud del coseno entre vectores.La función devuelve la lista de títulos recomendados.

In [22]:
# Generar una función para obtener recomendaciones por título
def recomendacion(juego):
    #Se busca el índice del juego en el DataFrame original (df). Este índice es utilizado para acceder a la fila correspondiente en la matriz de similitud.
    indice_juego = df[df["item_id"] == juego].index[0]
    
    #Cálculo de Similitudes: Se obtienen las distancias de similitud entre el juego de entrada y todos los demás juegos en el conjunto de datos. 
    distances = similitud[indice_juego]
    
    #Las distancias se ordenan de manera descendente, y se seleccionan los cinco juegos más similares (excluyendo el juego de entrada)
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    #Salida. Devuelve la lista de títulos recomendados.
    recommended_titles = [df.iloc[i[0]]['name'] for i in lista_juegos]
    
    return recommended_titles

In [24]:
# Aplicar la función a la columna 'item_id' y crear una nueva columna 'Recomendaciones'
df['Recomendaciones'] = df['item_id'].apply(recomendacion)

In [25]:
# Eliminar columnas innecesarias para disminuir el tamaño del archivo de salida
df.drop(columns=['name', 'specs'], inplace=True)
df

,item_id,Recomendaciones
0,761140,"[弹炸人2222, Uncanny Islands, Beach Rules, Planet..."
1,643980,"[Duelyst, Warhammer 40,000: Regicide, KROSMAGA..."
2,670290,"[Heroes of Havoc: Idle Adventures, Tactical Mo..."
3,767400,"[弹炸人2222, Uncanny Islands, Beach Rules, Planet..."
4,773570,"[Jam Session VR, The Trace, Caretaker Retribut..."
...,...,...
32128,773640,[Army of Tentacles: (Not) A Cthulhu Dating Sim...
32129,733530,"[Runespell: Overture, Rush for Glory, BoomTown..."
32130,610660,"[Drawn®: The Painted Tower, Tropico 4, The Bin..."
32131,658870,"[Fate of the World, Fate of the World: Tipping..."


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   item_id          32133 non-null  int32 
 1   Recomendaciones  32133 non-null  object
dtypes: int32(1), object(1)
memory usage: 376.7+ KB


In [27]:
# Guardar el DataFrame resultante en un nuevo archivo CSV
df.to_csv('recomienda_item_item.csv', index=False)